# GPT-2 Prompt Runner

This Colab notebook installs Hugging Face Transformers, loads a GPT-2 model, and lets you enter a free-form prompt. Run the cells from top to bottom, type your prompt when asked, and the model's continuation will be printed in the final cell.


In [ ]:
# Install the lightweight dependencies needed for GPT-2.
!pip install --quiet transformers torch accelerate


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

MODEL_NAME = "gpt2"
device = 0 if torch.cuda.is_available() else -1

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.config.pad_token_id = tokenizer.pad_token_id

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=device,
)

device_name = 'GPU' if device == 0 else 'CPU'
print(f"Loaded {MODEL_NAME} on {device_name}.")


In [ ]:
prompt = input("Enter a prompt for GPT-2: ").strip()
if not prompt:
    raise ValueError("Please enter a non-empty prompt.")

max_new_tokens = 120  # Adjust to control continuation length
outputs = generator(
    prompt,
    max_new_tokens=max_new_tokens,
    do_sample=True,
    temperature=0.8,
    top_p=0.95,
    num_return_sequences=1,
)

generated_text = outputs[0]["generated_text"]
print("\n=== Model Output ===")
print(generated_text)
